使用許諾条件

①	設計一次エネルギー消費量算定法に基づく計算方法をPythonによりコーディングし、Jupyter notebook形式で保存したファイル（以下、本ファイルと呼ぶ）の著作権は、一般社団法人日本サステナブル建築協会に帰属します。
Copyright©2019. 一般社団法人日本サステナブル建築協会
All rights reserved. 

②	本ファイルは、住戸の設計一次エネルギー消費量算定方法に係る「平成28年省エネルギー基準に準拠したエネルギー消費性能の評価に関する技術情報（住宅）2.1算定方法（第三章第一節 全般）」に示す計算方法に基づき、国土交通省国土技術政策総合研究所・国立研究開発法人建築研究所、株式会社ユーワークス、株式会社住環境計画研究所が作成したものです。

③	本ファイルは、計算方法に示す計算過程の確認及び調査研究等に活用することを目的として作成されたものであり、一般社団法人日本サステナブル建築協会は著作権について放棄はしませんが、使用する各人の自己責任において仕様を変更することを妨げるものではありません。ただし、本ファイルによる計算結果を住宅の省エネルギー基準で規定する設計一次エネルギー消費量の結果として公的な申請に用いることはできません。

④	本ファイルを変更するかしないかを問わず、以下の条件を満たす場合に限り、頒布および使用が許可されます。
1.	頒布物の付属ドキュメント等の資料に、上記の著作権表示、本条件一覧、および下記免責条項を含めること。
2.	書面による特別の許可なしに、本ファイルから派生した製品の宣伝または販売の促進に、一般社団法人日本サステナブル建築協会、国土技術政策総合研究所・国立研究開発法人建築研究所および株式会社ユーワークス、株式会社住環境計画研究所の名前またはコントリビューターの名前を使用してはならない。

⑤	本ファイルは、著作権者およびコントリビューターによって「現状のまま」提供されており、明示黙示を問わず、商業的な使用可能性、および特定の目的に対する適合性に関する暗黙の保証も含め、またそれに限定されない、いかなる保証もありません。著作権もコントリビューターも、事由のいかんを問わず、損害発生の原因いかんを問わず、かつ責任の根拠が契約であるか厳格責任であるか（過失その他の）不法行為であるかを問わず、仮にそのような損害が発生する可能性を知らされていたとしても、本ファイルの使用によって発生した（代用品または代用サービスの調達、使用の喪失、データの喪失、利益の喪失、業務の中断も含め、またそれに限定されない）直接損害、間接損害、偶発的な損害、特別損害、懲罰的損害、または結果損害について、一切の責任を負わないものとします。



In [ ]:
import nbimporter
import numpy as np

import s03_01_e_underfloor_temperature as algo

## 付録D　床下空間を経由して外気を導入する換気方式 / Appendix D Ventilation system which introduces outside air via underfloor space

### D.1 適用条件 / Applicable condition

省略

Omission

### D.2 外気を導入する床下の面積の算出方法 / Calculation method of the area under the floor which introduces outside air

省略

Omission

### D.3 床下空間を経由して外気を導入する換気方式による暖冷房負荷削減量 / The amount of heating and cooling load reduction by the ventilation system which introduces outside air via the space under the floor

#### D.3.1 記号及び単位 / Symbol and unit

省略

Omission

#### D.3.2 暖冷房負荷削減量 / Heating and cooling load reduction amount

$L'_{H,R,d,t,i} \leq 0$ の場合： 

For $L'_{H,R,d,t,i} \leq 0$:

$$\large
\begin{equation*}
    \Delta L'_{H,uf,d,t,i} = 0
\end{equation*}
$$

$0 < L'_{H,R,d,t,i}$ の場合： 

For $0 < L'_{H,R,d,t,i}$:

$$\large
\begin{equation*}
    \Delta L'_{H,uf,d,t,i} = \rho_{air} \times c_{p_{air}} \times V_{i} \times (\theta_{uf,d,t} - \theta_{in,d,t}) \times 10^{-3}\\ 
    - \rho_{air} \times c_{p_{air}} \times V_{i} \times (\theta_{ex,d,t} - \theta_{in,d,t}) \times 10^{-3}\\ 
    - U_s \times A_{s,ufvnt,i} \times (\theta_{in,d,t} - \theta_{uf,d,t}) \times 3.6 \times 10^{-3}\\
    + U_{s,vert} \times A_{s,ufvnt,i} \times (\theta_{in,d,t} - \theta_{ex,d,t}) \times H_{floor} \times 3.6 \times 10^{-3}
\end{equation*}
$$

In [ ]:
def calc_delta_L_dash_H_uf_d_t_i(i, A_A, A_MR, A_OR, A_HCZ_i, region, Q, r_A_ufvnt, underfloor_insulation, Theta_ex_d_t,
                                 L_dash_H_R_d_t_i, L_dash_CS_R_d_t_i):
    """ 当該住戸の暖冷房区画iの床下空間を経由して外気を導入する換気方式による暖房負荷削減量 (MJ/h) (1)

    :param i: 暖冷房区画の番号
    :type i: int
    :param A_A: 床面積の合計[m^2]
    :type A_A: float
    :param A_MR: 主たる居室の床面積[m^2]
    :type A_MR: float
    :param A_OR: その他の居室の床面積[m^2]
    :type A_OR: float
    :param A_HCZ_i: 暖冷房区画iの床面積 (m2)
    :type A_HCZ_i: float
    :param region: 省エネルギー地域区分
    :type region: int
    :param Q: 当該住戸の熱損失係数 (W/m2K)
    :type Q: float
    :param r_A_ufvnt: 当該住戸において、床下空間全体の面積に対する空気を供給する床下空間の面積の比 (-)
    :type r_A_ufvnt: float
    :param underfloor_insulation: 床下空間が断熱空間内である場合はTrue
    :type underfloor_insulation: bool
    :param Theta_ex_d_t: 外気温度 (℃)
    :type Theta_ex_d_t: ndarray
    :param L_dash_H_R_d_t_i: 標準住戸の負荷補正前の暖房負荷 (MJ/h)
    :type L_dash_H_R_d_t_i: ndarray
    :param L_dash_CS_R_d_t_i: 標準住戸の負荷補正前の冷房顕熱負荷 （MJ/h）
    :type L_dash_CS_R_d_t_i: ndarray
    :return: 当該住戸の暖冷房区画iの床下空間を経由して外気を導入する換気方式による暖房負荷削減量 (MJ/h)
    :rtype: ndarray
    """

    delta_L_dash_H_uf_d_t_i = np.zeros(24 * 365)

    if r_A_ufvnt is None or underfloor_insulation is None:
        return delta_L_dash_H_uf_d_t_i

    # ----------------------------------------
    # (1-1) L_dash_H_R_d_t_i <= 0
    # ----------------------------------------
    f1 = (L_dash_H_R_d_t_i[i - 1] <= 0)
    delta_L_dash_H_uf_d_t_i[f1] = 0

    # ----------------------------------------
    # (1-2) 0 < L_dash_H_R_d_t_i
    # ----------------------------------------
    f2 = (0 < L_dash_H_R_d_t_i[i - 1])

    # 空気の密度 (kg/m3)
    ro_air = get_ro_air()

    # 空気の比熱 (kJ/(kg・K))
    c_p_air = get_c_p_air()

    # 床の熱貫流率 (W/(m2・K))
    U_s = get_U_s()

    # 床の温度差係数 (-)
    H_floor = get_H_floor()

    # 室内温度 (℃)
    Theta_in_d_t = get_Theta_in_d_t('H')

    # 当該住戸の暖冷房区画iの外気を導入する床下空間に接する床の面積
    A_s_ufvnt_i = algo.calc_A_s_ufvnt_i(i, r_A_ufvnt, A_A, A_MR, A_OR)

    # 暖冷房負荷計算時に想定した床の熱貫流率
    U_s_vert = algo.get_U_s_vert(region, Q)

    # 当該住戸の1時間当たりの換気量 (m3/h) (4)
    V_A = get_V_A(A_A)

    # 当該住戸の暖冷房区画iの1時間当たりの換気量 (m3/h) (3)
    V_i = get_V_i(i, V_A, A_HCZ_i, A_MR, A_OR)

    # 床下温度及び地盤またはそれを覆う基礎の表面温度 (℃)
    Theta_uf_d_t, Theta_g_surf_d_t = algo.calc_Theta(
        region=region,
        A_A=A_A,
        A_MR=A_MR,
        A_OR=A_OR,
        Q=Q,
        r_A_ufvnt=r_A_ufvnt,
        underfloor_insulation=underfloor_insulation,
        Theta_sa_d_t=Theta_ex_d_t,
        Theta_ex_d_t=Theta_ex_d_t,
        V_sa_d_t_A=np.repeat(V_A, 24 * 365),
        H_OR_C='H',
        L_dash_H_R_d_t=L_dash_H_R_d_t_i,
        L_dash_CS_R_d_t=L_dash_CS_R_d_t_i
    )

    delta_L_dash_H_uf_d_t_i[f2] = (ro_air * c_p_air * V_i * (Theta_uf_d_t[f2] - Theta_in_d_t[f2]) * 10 ** (-3)
                                   - ro_air * c_p_air * V_i * (Theta_ex_d_t[f2] - Theta_in_d_t[f2]) * 10 ** (-3)
                                   - U_s * A_s_ufvnt_i * (Theta_in_d_t[f2] - Theta_uf_d_t[f2]) * 3.6 * 10 ** (-3)
                                   + U_s_vert * A_s_ufvnt_i
                                   * (Theta_in_d_t[f2] - Theta_ex_d_t[f2]) * H_floor * 3.6 * 10 ** (-3))

    return delta_L_dash_H_uf_d_t_i


$L'_{CS,R,d,t,i} \leq 0$ の場合： 

For $L'_{CS,R,d,t,i} \leq 0$:

$$\large
\begin{equation*}
    \Delta L'_{CS,uf,d,t,i} = 0
\end{equation*}
$$

$0 < L'_{CS,R,d,t,i}$ の場合： 

For $0 < L'_{CS,R,d,t,i}$:

$$\large
\begin{equation*}
    \Delta L'_{CS,uf,d,t,i} = - \rho_{air} \times c_{p_{air}} \times V_{i} \times (\theta_{uf,d,t} - \theta_{in,d,t}) \times 10^{-3}\\ 
    + \rho_{air} \times c_{p_{air}} \times V_{i} \times (\theta_{ex,d,t} - \theta_{in,d,t}) \times 10^{-3}\\ 
    + U_s \times A_{s,ufvnt,i} \times (\theta_{in,d,t} - \theta_{uf,d,t}) \times 3.6 \times 10^{-3}\\
    - U_{s,vert} \times A_{s,ufvnt,i} \times (\theta_{in,d,t} - \theta_{ex,d,t}) \times H_{floor} \times 3.6 \times 10^{-3}
\end{equation*}
$$

In [ ]:
def calc_delta_L_dash_CS_R_d_t_i(i, region, Q, r_A_ufvnt, underfloor_insulation, A_A, A_MR, A_OR, A_HCZ_i, Theta_ex_d_t,
                                 L_dash_H_R_d_t_i, L_dash_CS_R_d_t_i):
    """ 当該住戸の暖冷房区画iの床下空間を経由して外気を導入する換気方式による冷房顕熱負荷削減量 (MJ/h) (2)

    :param i: 暖冷房区画の番号
    :type i: int
    :param region: 省エネルギー地域区分
    :type region: int
    :param Q: 当該住戸の熱損失係数 (W/m2K)
    :type Q: float
    :param r_A_ufvnt: 当該住戸において、床下空間全体の面積に対する空気を供給する床下空間の面積の比 (-)
    :type r_A_ufvnt:
    :param underfloor_insulation: 床下空間が断熱空間内である場合はTrue
    :type underfloor_insulation:
    :param A_A: 床面積の合計 (m2)
    :type A_A: float
    :param A_MR: 主たる居室の床面積 (m2)
    :type A_MR: float
    :param A_OR: その他の居室の床面積 (m2)
    :type A_OR: float
    :param A_HCZ_i: 暖冷房区画iの床面積 (m2)
    :type A_HCZ_i: float
    :param Theta_ex_d_t: 外気温度 (℃)
    :type Theta_ex_d_t: ndarray
    :param L_dash_H_R_d_t_i: 標準住戸の負荷補正前の暖房負荷 (MJ/h)
    :type L_dash_H_R_d_t_i: ndarray
    :param L_dash_CS_R_d_t_i: 標準住戸の負荷補正前の冷房顕熱負荷 （MJ/h）
    :type L_dash_CS_R_d_t_i: ndarray
    :return: 当該住戸の暖冷房区画iの床下空間を経由して外気を導入する換気方式による冷房顕熱負荷削減量 (MJ/h) (2)
    :rtype: ndarray
    """

    delta_L_dash_CS_R_d_t_i = np.zeros(24 * 365)

    if r_A_ufvnt is None:
        return delta_L_dash_CS_R_d_t_i

    # ----------------------------------------
    # (2-1) L_dash_CS_R_d_t_i < 0
    # ----------------------------------------
    f1 = (L_dash_CS_R_d_t_i[i - 1] < 0)
    delta_L_dash_CS_R_d_t_i[f1] = 0

    # ----------------------------------------
    # (2-2) 0 < L_dash_CS_R_d_t_i
    # ----------------------------------------
    f2 = (0 < L_dash_CS_R_d_t_i[i - 1])

    # 空気の密度 (kg/m3)
    ro_air = get_ro_air()

    # 空気の比熱 (kJ/(kg・K))
    c_p_air = get_c_p_air()

    # 床の熱貫流率 (W/(m2・K))
    U_s = get_U_s()

    # 床の温度差係数 (-)
    H_floor = get_H_floor()

    # 当該住戸の1時間当たりの換気量 (m3/h) (4)
    V_A = get_V_A(A_A)

    # 当該住戸の暖冷房区画iの1時間当たりの換気量 (m3/h) (3)
    V_i = get_V_i(i, V_A, A_HCZ_i, A_MR, A_OR)

    # 室内温度 (℃)
    Theta_in_d_t = get_Theta_in_d_t('CS')

    # 床下温度及び地盤またはそれを覆う基礎の表面温度 (℃)
    Theta_uf_d_t, Theta_g_surf_d_t = algo.calc_Theta(
        region=region,
        A_A=A_A,
        A_MR=A_MR,
        A_OR=A_OR,
        Q=Q,
        r_A_ufvnt=r_A_ufvnt,
        underfloor_insulation=underfloor_insulation,
        Theta_sa_d_t=Theta_ex_d_t,
        Theta_ex_d_t=Theta_ex_d_t,
        V_sa_d_t_A=np.repeat(V_A, 24 * 365),
        H_OR_C='C',
        L_dash_H_R_d_t=L_dash_H_R_d_t_i,
        L_dash_CS_R_d_t=L_dash_CS_R_d_t_i
    )

    # 当該住戸の暖冷房区画iの外気を導入する床下空間に接する床の面積
    A_s_ufvnt_i = algo.calc_A_s_ufvnt_i(i, r_A_ufvnt, A_A, A_MR, A_OR)

    # 暖冷房負荷計算時に想定した床の熱貫流率
    U_s_vert = algo.get_U_s_vert(region, Q)

    delta_L_dash_CS_R_d_t_i[f2] = (- ro_air * c_p_air * V_i * (Theta_uf_d_t[f2] - Theta_in_d_t[f2]) * 10 ** (-3)
                                   + ro_air * c_p_air * V_i * (Theta_ex_d_t[f2] - Theta_in_d_t[f2]) * 10 ** (-3)
                                   + U_s * A_s_ufvnt_i * (Theta_in_d_t[f2] - Theta_uf_d_t[f2]) * 3.6 * 10 ** (-3)
                                   - U_s_vert * A_s_ufvnt_i * (
                                           Theta_in_d_t[f2] - Theta_ex_d_t[f2]) * H_floor * 3.6 * 10 ** (-3))

    return delta_L_dash_CS_R_d_t_i


$L'_{CS,R,d,t,i}$：日付$d$の時刻$t$における標準住戸の暖冷房区画$i$の負荷補正前の冷房顕熱負荷(MJ/h) / Cooling sensible heat load before load correction of heating / cooling section $ i $ of standard dwelling unit at time $ t $ of date $ d $  
$L'_{H,R,d,t,i}$：日付$d$の時刻$t$における標準住戸の暖冷房区画$i$の負荷補正前の暖房負荷(MJ/h) / Heating load before load compensation of heating / cooling section $ i $ of standard dwelling unit at time $ t $ of date $ d $   
$\Delta L'_{H,uf,d,t,i}$：日付$d$の時刻$t$における当該住戸の暖冷房区画$i$の床下空間を経由して外気を導入する換気方式による暖房負荷削減量(MJ/h) / The amount of heating load reduction by the ventilation system that introduces outside air via the floor space of the heating and cooling section $ i $ of the relevant dwelling unit at the time $ t $ of date $ d $   
$\Delta L'_{CS,uf,d,t,i}$：日付$d$の時刻$t$における当該住戸の暖冷房区画$i$の床下空間を経由して外気を導入する換気方式による冷房顕熱負荷削減量(MJ/h) / Cooling sensible heat load reduction amount by ventilation system which introduces outside air via underfloor space of heating and cooling section $ i $ of the dwelling unit at time $ t $ of date $ d $  
$\rho_{air}$： 空気の密度(kg/m<sup>3</sup>) / Air density   
$c_{p_{air}}$： 空気の比熱(kJ/(kg・K)) / Specific heat of air   
$V_{i}$：当該住戸の暖冷房区画$i$の 1 時間当たりの換気量(m<sup>3</sup>/h) / Ventilation rate per hour of heating / cooling section $ i $ of the relevant dwelling unit  
$U_s$：床の熱貫流率(W/(m<sup>2</sup>・K)) / Heat transmission coefficient of floor  
$U_{s,vert}$：暖冷房負荷計算時に想定した床の熱貫流率(W/(m<sup>2</sup>・K)) / Heat transmission coefficient of floor assumed when calculating heating and cooling load  
$A_{s,ufvnt,i}$：当該住戸の暖冷房区画$i$の空気を導入する床下空間に接する床の面積(m<sup>2</sup>) / Area of floor in contact with the space under the floor where air of heating and cooling section $ i $ of the dwelling unit concerned is introduced  
$\theta_{ex,d,t}$：日付$d$の時刻$t$における外気温度(℃) / Outside temperature at time $ t $ of date $ d $  
$\theta_{uf,d,t}$：日付$d$の時刻$t$における当該住戸の床下温度(℃) / Under floor temperature of the dwelling unit at time $ t $ of date $ d $  
$\theta_{in,d,t}$：日付$d$の時刻$t$における室内温度(℃) / Room temperature at time $ t $ with date $ d $  
$H_{floor}$：床の温度差係数(-) / Floor temperature difference coefficient  

空気の密度$\rho_{air}$は、1.20 kg/m<sup>3</sup>とする。空気の比熱$c_{p_{air}}$は、1.006 kJ/(kg・K)とする。床の熱貫流率$U_s$は、2.223 W/(m<sup>2</sup>・K)とする。床の温度差係数$H_{floor}$は、0.7 とする。日付$d$の時刻$t$における室内温度$\theta_{in,d,t}$は、暖房負荷削減量$\Delta L'_{H,uf,d,t,i}$の算定にあっては20℃、冷房顕熱負荷削減量$\Delta L'_{CS,uf,d,t,i}$の算定にあっては27℃とする。

The density of air $ \rho_ {air} $ is 1.20 kg / m 3. The specific heat $ c_ {p_ {air}} $ of air is 1.006 kJ / (kg · K). The heat transmission coefficient $ U_s $ of the bed is 2.223 W / (m <sup> 2 </sup> · K). The floor temperature difference coefficient $ H_ {floor} $ is 0.7. The room temperature $ \theta_ {in, d, t} $ at the time $ t $ of the date $ d $ is used to calculate the heating load reduction amount $ \Delta L '_ {H, uf, d, t, i} $ In the calculation of the cooling sensible heat load reduction amount $ \Delta L '_ {CS, uf, d, t, i} $, the temperature is assumed to be 27 ° C.

In [ ]:
def get_ro_air():
    """ 空気の密度 (kg/m3)

    :return: 空気の密度 (kg/m3)
    :rtype: float
    """
    return 1.20


def get_c_p_air():
    """ 空気の比熱 (kJ/(kg・K))

    :return: 空気の比熱 (kJ/(kg・K))
    :rtype: float
    """
    return 1.006


def get_U_s():
    """ 床の熱貫流率 (W/(m2・K))

    :return: 床の熱貫流率 (W/(m2・K))
    :rtype: float
    """
    return 2.223


def get_H_floor():
    """ 床の温度差係数 (-)

    :return: 床の温度差係数 (-)
    :rtype: float
    """
    return 0.7


def get_Theta_in_d_t(H_or_CS):
    """ 室内温度 (℃)

    :param H_or_CS: 計算対象('H' for 暖房負荷削減量, 'CS' for 冷房顕熱負荷削減量)
    :type H_or_CS: str
    :return: 室内温度 (℃)
    :rtype: ndarray
    """

    if H_or_CS == 'H':
        return np.repeat(20, 24 * 365)
    elif H_or_CS == 'CS':
        return np.repeat(27, 24 * 365)


#### D.3.3 換気量 / Ventilation volume

$$
\large
V_{i} = \left\{
\begin{array}{ll}
V_{A} \times \frac{A_{HCZ,i}}{A_{MR} + A_{OR}} & (i = 1～5 の場合)\\
0 & (i = 6～12 の場合)\\
\end{array}
\right.
$$

In [ ]:
def get_V_i(i, V_A, A_HCZ_i, A_MR, A_OR):
    """ 当該住戸の暖冷房区画iの1時間当たりの換気量 (m3/h) (3)

    :param i: 暖冷房区画の番号
    :type i: int
    :param V_A: 当該住戸の1時間当たりの換気量 (m3/h)
    :type V_A: float
    :param A_HCZ_i: 暖冷房区画iの床面積 (m2)
    :type A_HCZ_i: float
    :param A_MR: 主たる居室の床面積 (m2)
    :type A_MR: float
    :param A_OR: その他の居室の床面積 (m2)
    :type A_OR: float
    :return: 当該住戸の暖冷房区画iの1時間当たりの換気量 (m3/h)
    :rtype: float
    """

    if i in [1, 2, 3, 4, 5]:
        return V_A * (A_HCZ_i / (A_MR + A_OR))
    elif i in [6, 7, 8, 9, 10, 11, 12]:
        return 0
    else:
        raise ValueError(i)


$$
\large
V_{A} = A_{A} \times H_{R} \times N
$$

In [ ]:
def get_V_A(A_A):
    """ 当該住戸の1時間当たりの換気量 (m3/h) (4)

    :param A_A: 床面積の合計 (m2)
    :type A_A: float
    :return: 当該住戸の1時間当たりの換気量 (m3/h)
    :rtype: float
    """

    # 参照天井高さ (m)
    H_R = get_H_R()

    # 換気回数 (回/h)
    N = get_N()

    # 当該住戸の1時間当たりの換気量 (4)
    V_A = A_A * H_R * N

    return V_A

$A_{A}$：床面積の合計(m<sup>2</sup>) / Total floor area  
$A_{HCZ,i}$：暖冷房区画$i$の床面積(m<sup>2</sup>) / Floor area of heating / cooling section $ i $  
$A_{MR}$：主たる居室の床面積(m<sup>2</sup>) / Floor area of main room  
$A_{OR}$：その他の居室の床面積(m<sup>2</sup>) / Floor area of other rooms  
$H_{R}$：参照天井高さ(m) / Reference ceiling height  
$N$：換気回数(1/h) / Ventilation frequency  
$V_{A}$：当該住戸の 1 時間当たりの換気量(m<sup>3</sup>/h) / Ventilation rate per hour of the dwelling unit concerned  
$V_{i}$：当該住戸の暖冷房区画$i$の1時間当たりの換気量(m<sup>3</sup>/h) / Ventilation rate per hour of heating / cooling section $ i $ of the relevant dwelling unit  

参照天井高さ$H_{R}$は、2.4 m とする。換気回数$N$は、0.5 回/h とする。 

The reference ceiling height $ H_{R} $ is 2.4 m. The ventilation frequency $ N $ shall be 0.5 times / h.

In [ ]:
def get_H_R():
    """ 参照天井高さ (m)

    :return: 参照天井高さ (m)
    :rtype: float
    """
    return 2.4


def get_N():
    """ 換気回数 (回/h)

    :return: 換気回数 (回/h)
    :rtype: float
    """
    return 0.5
